In [8]:
from biothings_explorer.user_query_dispatcher import SingleEdgeQueryDispatcher
import numpy as np
import pandas as pd
import networkx as nx

In [9]:
seqd = SingleEdgeQueryDispatcher(input_cls='Gene',
                                 output_cls='ChemicalSubstance',
                                 input_id='NCBIGene',
                                 values='1017')
                                 


In [10]:
seqd.query()

In [12]:
len(seqd.G)

859

In [12]:
print(seqd.G.number_of_edges())

1215


In [46]:
for i in seqd.G.nodes:
    print(i, seqd.G.degree(i), seqd.G.number_of_edges('NCBIGene:1017',i))

NCBIGene:1017 1215 0
CHEMBL1197513 1 1
CHEBI:52214 1 1
CHEBI:36080 1 1
CHEBI:33252 1 1
CHEBI:35222 2 2
CHEMBL2163995 1 1
CHEMBL1964246 1 1
CHEMBL213713 1 1
CHEMBL272833 1 1
CHEMBL259838 1 1
CHEMBL257831 1 1
CHEMBL3354187 1 1
CHEMBL1964244 1 1
CHEMBL455195 1 1
CHEMBL405145 1 1
CHEMBL1964259 1 1
CHEMBL1761791 1 1
CHEMBL3642647 1 1
CHEMBL411426 1 1
CHEMBL3655765 1 1
CHEMBL3298984 1 1
CHEMBL79498 1 1
CHEMBL255463 1 1
CHEMBL485618 1 1
CHEMBL181114 1 1
CHEMBL424696 1 1
CHEMBL186288 1 1
CHEMBL488085 1 1
CHEMBL265903 1 1
CHEMBL415471 1 1
CHEMBL215086 1 1
CHEMBL496785 1 1
CHEMBL1171949 1 1
CHEMBL2158843 1 1
CHEMBL1964242 1 1
CHEMBL103285 1 1
CHEMBL295484 1 1
CHEMBL3357950 1 1
CHEMBL291324 1 1
CHEMBL79356 1 1
CHEMBL515001 1 1
CHEMBL260163 1 1
CHEMBL268507 1 1
CHEMBL255263 1 1
CHEMBL485619 1 1
CHEMBL3648117 1 1
CHEMBL520187 1 1
CHEMBL361833 1 1
CHEMBL476993 1 1
CHEMBL3644022 1 1
CHEMBL1964260 1 1
CHEMBL3648114 1 1
CHEMBL603097 1 1
CHEMBL509012 1 1
CHEMBL3648123 1 1
CHEMBL187750 1 1
CHEMBL260103 1

4-{5-[(1Z)-1-(2-IMINO-4-OXO-1,3-THIAZOLIDIN-5-YLIDENE)ETHYL]-2-FURYL}BENZENESULFONAMIDE 2 2
N-[4-(2,4-DIMETHYL-THIAZOL-5-YL)-PYRIMIDIN-2-YL]-N',N'-DIMETHYL-BENZENE-1,4-DIAMINE 2 2
6-(3,4-DIHYDROXYBENZYL)-3-ETHYL-1-(2,4,6-TRICHLOROPHENYL)-1H-PYRAZOLO[3,4-D]PYRIMIDIN-4(5H)-ONE 2 2
6-(3-AMINOPHENYL)-N-(TERT-BUTYL)-2-(TRIFLUOROMETHYL)QUINAZOLIN-4-AMINE 2 2
2-(4-(AMINOMETHYL)PIPERIDIN-1-YL)-N-(3_CYCLOHEXYL-4-OXO-2,4-DIHYDROINDENO[1,2-C]PYRAZOL-5-YL)ACETAMIDE 2 2
1-(3-(2,4-DIMETHYLTHIAZOL-5-YL)-4-OXO-2,4-DIHYDROINDENO[1,2-C]PYRAZOL-5-YL)-3-(4-METHYLPIPERAZIN-1-YL)UREA 2 2
4-{[5-(CYCLOHEXYLMETHOXY)[1,2,4]TRIAZOLO[1,5-A]PYRIMIDIN-7-YL]AMINO}BENZENESULFONAMIDE 2 2
4-{[5-(CYCLOHEXYLAMINO)[1,2,4]TRIAZOLO[1,5-A]PYRIMIDIN-7-YL]AMINO}BENZENESULFONAMIDE 2 2
4-({5-[(4-AMINOCYCLOHEXYL)AMINO][1,2,4]TRIAZOLO[1,5-A]PYRIMIDIN-7-YL}AMINO)BENZENESULFONAMIDE 2 2
4-{[5-(CYCLOHEXYLOXY)[1,2,4]TRIAZOLO[1,5-A]PYRIMIDIN-7-YL]AMINO}BENZENESULFONAMIDE 2 2
CAN-508 2 2
N-[3-(1H-BENZIMIDAZOL-2-YL)-1H-PYRAZOL-4-YL]BENZAM

# Filter 1
### Filter by number of edges overlapping between nodes

In [2]:
def filter_node_degree(G, count=50, filt='NodeDegree'): #takes input G as networkX graph

    source = [x for x,y in G.nodes(data=True) if y['level']==1][0]
    degrees = [[],[]]
    for node in G.nodes:
        if node != source:
            degrees[0].append(node)
            degrees[1].append(G.degree(node))

    data = {'node':degrees[0], 'degree':degrees[1]}
    deg_count = pd.DataFrame(data=data)
    deg_count.sort_values(by='degree', inplace=True, ascending=False)

    filtered = list(deg_count.head(count)['node']) + [source]
    subG = G.subgraph(filtered)

    for i,node in enumerate(filtered[:-1], start=1):
        subG.nodes.data()[node]['filteredBy'] = filt
        subG.nodes.data()[node]['rank'] = i

    return subG

In [163]:
x = filter_node_degree(seqd.G)


In [210]:
for i,node in enumerate(x.nodes):
    print(i,node,x.degree(node),x.nodes.data()[node]['rank'], '\n')

0 DINACICLIB 11 5 

1 AT-7519 7 9 

2 STAUROSPORINE 7 10 

3 ALVOCIDIB 38 2 

4 OLOMOUCINE 12 4 

5 SELICICLIB 9 6 

6 NCBIGene:1017 129 1 

7 ROSCOVITINE 8 7 

8 UCN-01 30 3 

9 RONICICLIB 7 8 



In [233]:
l = []
for i in range(len(x.nodes)):
    for node in x.nodes:
        if x.nodes.data()[node]['rank'] == i+1:
            l.append(node)
l

['NCBIGene:1017',
 'ALVOCIDIB',
 'UCN-01',
 'OLOMOUCINE',
 'DINACICLIB',
 'SELICICLIB',
 'ROSCOVITINE',
 'RONICICLIB',
 'AT-7519',
 'STAUROSPORINE']

In [206]:
x.nodes(3)

NodeDataView({'DINACICLIB': None, 'AT-7519': None, 'STAUROSPORINE': None, 'ALVOCIDIB': None, 'OLOMOUCINE': None, 'SELICICLIB': None, 'NCBIGene:1017': None, 'ROSCOVITINE': None, 'UCN-01': None, 'RONICICLIB': None}, data=3)

In [143]:
assert seqd.G['NCBIGene:1017']['AT-7519']== x['NCBIGene:1017']['AT-7519']

In [180]:
import unittest

In [ ]:
class TestFilterEdges(unittest.TestCase):

    # test for count values
    def test_count_values(self):
        counts = [10, 20, 40, 50, 100, 1050]

        seqd = SingleEdgeQeryDispatcher(output_cls='GenomicEntity',
                                        input_cls='Gene',
                                        input_id='HGNC',
                                        values='7890')
        seqd.query()
        for count in counts:
            newG = filter_node_degree(seqd.G, count)
            self.assertEqual(len(newG.nodes), 2)

# Filter 2
### Filter by type of relation

Essentially takes in graph input, the relation we are looking for
and returns a graph with only the edges with said relation, ranked then by degree

In [156]:
seqd = SingleEdgeQueryDispatcher(input_cls='Gene',
                                 output_cls='ChemicalSubstance',
                                 input_id='NCBIGene',
                                 values='1017')
seqd.query()

In [412]:
x = []
for i in seqd.G.edges:
    x.append(seqd.G[i[0]][i[1]][0]['label'])
    
x = set(x)
len(seqd.G.nodes)

3

In [3]:
def filter_label(G, label, count=50):
    
    val_edges = []
    for edge in G.edges:
        if G[edge[0]][edge[1]][edge[2]]['label'] in label:
            val_edges.append(edge)
    
    subG = G.edge_subgraph(val_edges)
    subG = filter_node_degree(subG, count, 'EdgeLabel')
        
    return subG

In [158]:
labels = ['related_to', 'negatively_regulated_by']
subG = filter_label(seqd.G, labels, 10)

In [404]:
len(seqd.G.edges)

1215

In [359]:
val = []
label = 'related_to'
for edge in seqd.G.edges:
    if seqd.G[edge[0]][edge[1]][0]['label'] == label:
        val.append(edge)
        
sG = seqd.G.edge_subgraph(val)

len(val)


582

In [63]:
seqd2 = SingleEdgeQueryDispatcher(output_cls='GenomicEntity',
                                         input_cls='ChemicalSubstance',
                                         pred="related_to",
                                         input_id='CHEBI',
                                         values='CHEBI:28640')
seqd2.query()

# Filter 3: Co-occurrence
### Using NIH MRCOC co-occurrence files, query MRCOC API

In [2]:
import time
import requests

In [4]:

# this is the one used
def filter_co_occur(G, count=50):
        
    # helper funcs
    def get_ids(node):
        ids = []
        try:
            ids.append(G.nodes[node]['equivalent_ids']['MESH'])
            ids.append(G.nodes[node]['equivalent_ids']['UMLS'])
        except:
            pass

        ids = [i for sub in ids for i in sub] # flatten and get rid of set()

        return 0 if len(ids) == 0 else ids

    def make_combo(id1, id2):
        combos = ['-'.join([i,j]) for i in id1 for j in id2]
        combos += ['-'.join([j,i]) for i in id1 for j in id2]

        return combos
    
    
    unique_edges = []
    source = [x for x,y in G.nodes(data=True) if y['level']==1][0]
    for node in G.nodes:
        if node != source:
            unique_edges.append([source,node])

    num_combs, combos = [], []
    src_id = get_ids(source)
    for edge in unique_edges:
        tar_id = get_ids(edge[1])

        if (src_id == 0) | (tar_id == 0):
            edge.insert(0, 100)
        else:
            combo = make_combo(src_id, tar_id)
            combos.append(combo)
            num_combs.append(len(combo))

    combos = [i for j in combos for i in j]
    chunks = [combos[x:x+1000] for x in range(0,len(combos),1000)]
    x = []
    for chunk in chunks:
        x += requests.post('https://biothings.ncats.io/mrcoc/query', json={'scopes':'combo', 'q': chunk}).json()

    end, i = 0, 0
    for edge in unique_edges:
        if isinstance(edge[0],int):
            continue
        start = end
        end += num_combs[i]
        for query in x[start:end]:
            if not 'notfound' in query:
                edge.insert(0, query['ngd_overall'])
                break
        if not isinstance(edge[0], float):
            edge.insert(0, 200)
        i+=1

    results = sorted(unique_edges)[:count]
    filtered = [i[2] for i in results] + [source]
    subG = G.subgraph(filtered)
    
    for i,res in enumerate(results, start=1):
        subG.nodes[res[2]]['rank'] = i
        subG.nodes[res[2]]['filteredBy'] = 'CoOccurrence'
        subG.nodes[res[2]]['ngd_overall'] = res[0]

    return subG

In [105]:
def filter_co_occur(G, count=50):
        
    # helper funcs
    def get_ids(node):
        ids = []
        try:
            ids.append(G.nodes[node]['equivalent_ids']['MESH'])
            ids.append(G.nodes[node]['equivalent_ids']['UMLS'])
        except:
            pass

        ids = [i for sub in ids for i in sub] # flatten and get rid of set()

        return 0 if len(ids) == 0 else ids

    def make_combo(id1, id2):
        combos = ['-'.join([i,j]) for i in id1 for j in id2]
        combos += ['-'.join([j,i]) for i in id1 for j in id2]

        return combos
    
    
    unique_edges = []
    for edge in G.edges:
        if [edge[0], edge[1]] in unique_edges:
            continue
        else:
            unique_edges.append([edge[0], edge[1]])

    num_combs, combos = [], []
    for edge in unique_edges:
        id1 = get_ids(edge[0])
        id2 = get_ids(edge[1])

        if (id1 == 0) | (id2 == 0):
            edge.insert(0, 100)
        else:
            combo = make_combo(id1, id2)
            combos.append(combo)
            num_combs.append(len(combo))

    combos = [i for j in combos for i in j]
    chunks = [combos[x:x+1000] for x in range(0,len(combos),1000)]
    x = []
    for chunk in chunks:
        x += requests.post('https://biothings.ncats.io/mrcoc/query', json={'scopes':'combo', 'q': chunk}).json()

    end, i = 0, 0
    for edge in unique_edges:
        if isinstance(edge[0],int):
            continue
        start = end
        end += num_combs[i]
        for query in x[start:end]:
            if not 'notfound' in query:
                edge.insert(0, query['ngd_overall'])
                break
        if not isinstance(edge[0], float):
            edge.insert(0, 200)
        i+=1

    results = sorted(unique_edges)[:count]
    filtered = list(set([i[1] for i in results] + [i[2] for i in results]))
    subG = G.subgraph(filtered)
    
    for i,res in enumerate(results, start=1):
        subG.nodes[res[2]]['rank'] = i
        subG.nodes[res[2]]['filteredBy'] = 'CoOccurrence'
        subG.nodes[res[2]]['ngd_overall'] = res[0]

    return subG

In [9]:
seqd = SingleEdgeQueryDispatcher(input_cls='Disease', output_cls='Disease', input_id='MESH', values='D000755')
seqd.query()

In [106]:
s1 = time.time()
G = filter_co_occur(seqd.G)
e1 = time.time()
s2 = time.time()
G2 = filter_co_occur2(seqd.G)
e2 = time.time()
print('G1 time: {}\tG2 time: {}'.format(e1-s1,e2-s2))

G1 time: 1.3584139347076416	G2 time: 1.3247249126434326


In [201]:
unique_edges = []
for edge in seqd.G.edges:
    if [edge[0], edge[1]] in unique_edges:
        continue
    else:
        unique_edges.append([edge[0], edge[1]])

In [202]:
num_combs, combos = [], []
for edge in unique_edges:
    id1 = get_ids(edge[0])
    id2 = get_ids(edge[1])

    if (id1 == 0) | (id2 == 0):
        edge.insert(0, 100)
    else:
        combo = make_combo(id1, id2)
        combos.append(combo)
        num_combs.append(len(combo))

In [203]:
combos = [i for j in combos for i in j]
chunks = [combos[x:x+1000] for x in range(0,len(combos),1000)]

In [204]:
x = []
for chunk in chunks:
    x+= requests.post('https://biothings.ncats.io/mrcoc/query', json={'scopes':'combo', 'q': chunk}).json()

In [205]:
end, i = 0, 0
for edge in unique_edges:   
    if isinstance(edge[0],int):
        continue
        
    start = end
    end += num_combs[i]
    for query in x[start:end]:
        if not 'notfound' in query:
            edge.insert(0, query['ngd_overall'])
            print(edge)
            break
    if not isinstance(edge[0], float):
        edge.insert(0, 200)   
    i+=1

[0.3786116229133875, 'MESH:D000755', 'ALPHA THALASSEMIA']
[0.3520097217194215, 'MESH:D000755', 'ANEMIA']
[0.4193873106695017, 'MESH:D000755', 'ANEMIA, HEMOLYTIC']
[0.7189390880973388, 'MESH:D000755', 'ANKYLOSIS']
[0.8365177066376366, 'MESH:D000755', 'CARDIAC ARRHYTHMIA']
[0.6780323851612118, 'MESH:D000755', 'ARTHRITIS']
[0.6750172809472789, 'MESH:D000755', 'ASTHMA']
[0.7177989989115081, 'MESH:D000755', 'BACK PAIN']
[0.47847706925878947, 'MESH:D000755', 'CEREBRAL INFARCTION']
[0.6397025858996038, 'MESH:D000755', 'BENIGN RECURRENT INTRAHEPATIC CHOLESTASIS']
[0.7634033302810889, 'MESH:D000755', 'DISEASE OR DISORDER']
[0.6320661320820271, 'MESH:D000755', 'BRIGHT DISEASE']
[0.6155801802017089, 'MESH:D000755', 'HEADACHE']
[0.6029340682160718, 'MESH:D000755', 'NONSYNDROMIC GENETIC DEAFNESS']
[0.7488520180595014, 'MESH:D000755', 'HEREDITARY HYPERBILIRUBINEMIA']
[0.4658974351897532, 'MESH:D000755', 'PULMONARY HYPERTENSION']
[0.7182471219773823, 'MESH:D000755', 'HYPERTROPHY']
[0.3925935858047931

In [212]:
for i in range(len(pre_res)):
    assert pre_res[i] == unique_edges[i]

# Filter 4: by number of unique APIs supporting association

Testing modules:
* count is correct (50 returned + source)
* all have 'filteredBy' and 'rank'
* insure 'rank' is correct

In [5]:
def filter_api(G, count=50):
    source = [x for x,y in G.nodes(data=True) if y['level']==1][0]
    counts = []
    for node in G.nodes:
        if node == source:
            continue
        apis = set()
        for edge in G[source][node]:  
            apis.add(G[source][node][edge]['info']['$api'])
        counts.append([len(apis), node])

    counts = sorted(counts, reverse=True)[:count]
    filtered = [i[1] for i in counts] + [source]
    subG = G.subgraph(filtered)

    for i,node in enumerate(filtered[:-1], start=1):
        subG.nodes.data()[node]['filteredBy'] = 'UniqueAPIs'
        subG.nodes.data()[node]['rank'] = i
    return subG

In [7]:
seqd = SingleEdgeQueryDispatcher(input_cls='Gene',
                                 output_cls='ChemicalSubstance',
                                 input_id='NCBIGene',
                                 values='1017')
seqd.query()

In [7]:
seqd.G['NCBIGene:1017']['CHEBI:35222']

AtlasView({0: {'info': {'CHEBI': 'CHEBI:35222', '$api': 'Automat CORD19 Scigraph API', '$source': 'scigraph', '@type': 'ChemicalSubstance'}, 'label': 'related_to', 'source': 'scigraph'}, 1: {'info': {'@type': 'ChemicalSubstance', 'pmc': ['PMC6522292'], 'CHEBI': 'CHEBI:35222', '$api': 'CORD Gene API', '$source': 'Translator Text Mining Provider'}, 'label': 'related_to', 'source': 'Translator Text Mining Provider'}})

In [10]:
G = filter_api(seqd.G)

In [16]:
for x,y in G.nodes(data=True):
    print(x,y.keys())

VENOUS INSUFFICIENCY (DISEASE) dict_keys(['identifier', 'type', 'level', 'equivalent_ids', 'filteredBy', 'rank'])
OSTEOMYELITIS dict_keys(['identifier', 'type', 'level', 'equivalent_ids', 'filteredBy', 'rank'])
OSTEOPENIA dict_keys(['identifier', 'type', 'level', 'equivalent_ids', 'filteredBy', 'rank'])
THALASSEMIA dict_keys(['identifier', 'type', 'level', 'equivalent_ids', 'filteredBy', 'rank'])
SYSTEMIC DISEASE dict_keys(['identifier', 'type', 'level', 'equivalent_ids', 'filteredBy', 'rank'])
THROMBOPHILIA dict_keys(['identifier', 'type', 'level', 'equivalent_ids', 'filteredBy', 'rank'])
WATERHOUSE-FRIDERICHSEN SYNDROME dict_keys(['identifier', 'type', 'level', 'equivalent_ids', 'filteredBy', 'rank'])
OBSTRUCTIVE SLEEP APNEA SYNDROME dict_keys(['identifier', 'type', 'level', 'equivalent_ids', 'filteredBy', 'rank'])
TUBERCULOMA dict_keys(['identifier', 'type', 'level', 'equivalent_ids', 'filteredBy', 'rank'])
WHEEZING dict_keys(['identifier', 'type', 'level', 'equivalent_ids', 'filter

In [17]:
seqd = SingleEdgeQueryDispatcher(input_cls='Disease',
                                         input_id='MONDO',
                                         output_cls='PhenotypicFeature',
                                         pred='related_to',
                                         values='MONDO:0010997')
seqd.query()

In [18]:
seqd.G.nodes

NodeView(('MONDO:MONDO:0010997', 'HP:0002548', 'HP:0002544', 'HP:0002530', 'HP:0002529', 'HP:0002528', 'HP:0002527', 'HP:0002451', 'HP:0002439', 'HP:0002304', 'HP:0200147', 'HP:0002185', 'HP:0002141', 'HP:0002068', 'HP:0002067', 'HP:0002063', 'HP:0002015', 'HP:0000743', 'HP:0000741', 'HP:0000651', 'HP:0000643', 'HP:0000622', 'HP:0000605', 'HP:0000511', 'HP:0007164', 'HP:0007158', 'HP:0007086', 'HP:0030953', 'HP:0001260', 'HP:0001332', 'HP:0100710', 'HP:0000613', 'HP:0000658', 'HP:0001300', 'HP:0006921', 'HP:0001337', 'HP:0001268', 'HP:0002172', 'HP:0000514', 'HP:0007256', 'HP:0000570', 'HP:0000633', 'HP:0000737', 'HP:0002171', 'HP:0002300', 'HP:0002354', 'HP:0002174', 'HP:0000006', 'HP:0003581', 'HP:0001425'))

In [21]:
r = [0]*50

In [23]:
len(r)

50

__init__ tests

In [5]:
class Filter():

    def __init__(self, G, filt=None, label=None, count=50):
        self.G = G
        self.filter = filt
        self.count = count
        self.label = label

    def filter_results(self):
        if not self.filter:
            return self.G
        if self.filter == 'NodeDegree':
            return filter_node_degree(self.G, self.count)
        if self.filter == 'EdgeLabel':
            if not self.label:
                print('Please include a label for this filter')
            else:
                return filter_label(self.G, self.label, self.count)
        if self.filter == 'CoOccurrence':
            return filter_co_occur(self.G, self.count)
        if self.filter == 'UniqueAPIs':
            return filter_api(self.G, self.count)
        return self.G

In [45]:
filt = Filter(seqd.G, 'EdgeLabel', label='related_to').filter_results()

In [51]:
for x in filt.edges.data():
    print(x[2]['label'])

related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to
related_to

In [5]:
from ..user_query_dispatcher import SingleEdgeQueryDispatcher

ImportError: attempted relative import with no known parent package

In [6]:
seqd = SingleEdgeQueryDispatcher(input_cls='Gene',
                                 output_cls='ChemicalSubstance',
                                 input_id='NCBIGene',
                                 values='1017',
                                 filter={})

NameError: name 'Registry' is not defined

In [2]:
x = ' '
if not x:
    print('y')